CSV ──(Pandas)──> Tabela ──(SpaCy + NLTK)──> Processamento ──(Pandas)──> Nova Coluna ──(Joblib)──> Arquivo Final

Parte 1: Pandas & Numpy (A Base de Dados)

1. Pandas & Numpy: "A Bancada e a Balança" 

Eles não alteram o significado do texto, eles apenas seguram e medem os dados.

    Onde entra: No início (carregar) e no fim (organizar).
    Função:
        Pandas: É a bandeja onde todos os ingredientes (textos) ficam organizados em linhas e colunas. Sem ele, teríamos variáveis soltas voando.
        Numpy: É a calculadora. Se você quiser saber a média das notas ou o desvio padrão, ele entra em ação.
    No Código: df['texto'] (Pandas segura o dado para o Spacy pegar).

In [1]:
import pandas as pd
import numpy as np

# 1. Criando um Dataset Fictício (Pandas)
dados = {
    'id': [1, 2, 3, 4],
    'texto': [
        "O produto chegou quebrado e a caixa rasgada!!!",
        "Amei muito, entrega super rápida e segura.",
        "Péssimo atendimento, nunca mais compro.",
        "O produto é bom, mas a entrega demorou."
    ],
    'nota': [1, 5, 1, 3]
}

df = pd.DataFrame(dados)

# 2. Usando Numpy para estatísticas rápidas
media_notas = np.mean(df['nota'])
desvio_padrao = np.std(df['nota'])

print(f"Média das notas: {media_notas}")
print("--- Tabela Original ---")
display(df) # No Jupyter, use display() em vez de print() para ficar bonito

Média das notas: 2.5
--- Tabela Original ---


,id,texto,nota
0,1,O produto chegou quebrado e a caixa rasgada!!!,1
1,2,"Amei muito, entrega super rápida e segura.",5
2,3,"Péssimo atendimento, nunca mais compro.",1
3,4,"O produto é bom, mas a entrega demorou.",3


Parte 2: NLTK (A Limpeza Manual)

2. NLTK: "O Bibliotecário (Consultor)" 

No nosso fluxo moderno, o NLTK atuou como um fornecedor de referência.

    Onde entra: No meio do processamento, como apoio.
    Função: Nós não usamos o NLTK para processar o texto pesado (embora ele possa). Nós o usamos para consultar a lista de Stopwords.
    Analogia: O Spacy pergunta: "Ei NLTK, 'para' é uma palavra importante?". O NLTK consulta seu livro e diz: "Não, pode jogar fora".
    No Código: if token.text not in lista_stopwords (A lista veio do NLTK).

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixando recursos necessários
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

# Carregando stopwords em português
lista_stopwords = stopwords.words('portuguese')

def limpar_texto(texto):
    # 1. Tokeniza (corta em palavras)
    palavras = word_tokenize(texto.lower(), language='portuguese')
    
    # 2. Filtra (só deixa o que NÃO for stopword e for letra)
    palavras_limpas = [p for p in palavras if p not in lista_stopwords and p.isalpha()]
    
    return " ".join(palavras_limpas)

# Aplicando a função na coluna 'texto' do Pandas
df['texto_limpo'] = df['texto'].apply(limpar_texto)

print("--- Tabela com Texto Limpo ---")
display(df[['texto', 'texto_limpo']])

--- Tabela com Texto Limpo ---


[nltk_data] Downloading package punkt_tab to /home/matheus-
[nltk_data]     sales/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/matheus-
[nltk_data]     sales/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/matheus-
[nltk_data]     sales/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,texto,texto_limpo
0,O produto chegou quebrado e a caixa rasgada!!!,produto chegou quebrado caixa rasgada
1,"Amei muito, entrega super rápida e segura.",amei entrega super rápida segura
2,"Péssimo atendimento, nunca mais compro.",péssimo atendimento nunca compro
3,"O produto é bom, mas a entrega demorou.",produto bom entrega demorou


Parte 3: Spacy (A Inteligência)

3. Spacy: "O Chef (Processador)" 

Ele é quem coloca a mão na massa e transforma o ingrediente.

    Onde entra: No núcleo do loop de processamento.
    Função: Ele lê o texto, entende a gramática, identifica verbos e converte para o infinitivo (Lemma). É a biblioteca mais "pesada" e inteligente do fluxo.
    Analogia: Ele pega a "batata suja" (texto bruto), descasca, corta e devolve a "batata pronta" (lemma).
    No Código: doc = nlp(texto) e token.lemma_.

In [4]:
import spacy

# Carregando o cérebro em português
nlp = spacy.load("pt_core_news_sm")

def extrair_lemmas(texto):
    doc = nlp(texto)
    # Pega o lemma de cada palavra
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

# Criando uma coluna nova com os Lemmas
df['texto_lemma'] = df['texto'].apply(extrair_lemmas)

print("--- Comparação: Original vs Lemma ---")
# Note como "chegou" vira "chegar" e "compro" vira "comprar"
display(df[['texto', 'texto_lemma']])

--- Comparação: Original vs Lemma ---


,texto,texto_lemma
0,O produto chegou quebrado e a caixa rasgada!!!,o produto chegar quebrar e o caixa rasgar ! ! !
1,"Amei muito, entrega super rápida e segura.","Amei muito , entregar super rápido e seguro ."
2,"Péssimo atendimento, nunca mais compro.","péssimo atendimento , nunca mais compro ."
3,"O produto é bom, mas a entrega demorou.","o produto ser bom , mas o entrega demorar ."


Parte 4: Copy (Segurança de Dados)

4. Copy: "O Seguro de Vida" 

Ele atua nos bastidores para evitar acidentes.

    Onde entra: Antes de qualquer alteração drástica em dicionários ou listas complexas.
    Função: Garante que, se o Chef queimar a comida na panela nova, a panela velha ainda tenha a comida original intacta.
    No Código: copy.deepcopy().

In [5]:
import copy

# Imagine que temos uma configuração da nossa IA
config_original = {
    "modelo": "v1",
    "params": ["a", "b"]
}

# JEITO ERRADO (Referência)
config_teste = config_original
config_teste["modelo"] = "v2"
# Isso altera a original também! (Perigo!)

# JEITO CERTO (Deep Copy)
config_segura = copy.deepcopy(config_original)
config_segura["modelo"] = "v3"

print(f"Original: {config_original['modelo']}") # Ainda é v2 (por causa do erro acima)
print(f"Segura: {config_segura['modelo']}")     # É v3, e não afetou ninguém

Original: v2
Segura: v3


Parte 5: Joblib (Salvando o Trabalho)

5. Joblib: "O Freezer" 

Ele congela o trabalho pronto para ser consumido depois.

    Onde entra: No final absoluto.
    Função: Salvar a memória RAM (que apaga quando desliga o PC) no Disco Rígido (que é permanente). Ele é muito mais eficiente que salvar em .txt ou .csv porque ele salva o objeto Python inteiro.
    No Código: joblib.dump() e joblib.load().

In [6]:
import joblib

# 1. Salvando o DataFrame processado num arquivo
joblib.dump(df, 'meus_dados_processados.pkl')
print("Arquivo salvo com sucesso!")

# --- Imagine que você fechou o computador e voltou amanhã ---

# 2. Carregando de volta
df_recuperado = joblib.load('meus_dados_processados.pkl')

print("--- Dados Recuperados do Arquivo ---")
display(df_recuperado.head())

Arquivo salvo com sucesso!
--- Dados Recuperados do Arquivo ---


,id,texto,nota,texto_limpo,texto_lemma
0,1,O produto chegou quebrado e a caixa rasgada!!!,1,produto chegou quebrado caixa rasgada,o produto chegar quebrar e o caixa rasgar ! ! !
1,2,"Amei muito, entrega super rápida e segura.",5,amei entrega super rápida segura,"Amei muito , entregar super rápido e seguro ."
2,3,"Péssimo atendimento, nunca mais compro.",1,péssimo atendimento nunca compro,"péssimo atendimento , nunca mais compro ."
3,4,"O produto é bom, mas a entrega demorou.",3,produto bom entrega demorou,"o produto ser bom , mas o entrega demorar ."


In [7]:
def processamento_completo(texto):
    # 1. Spacy faz o trabalho pesado (Lemmatization)
    doc = nlp(texto.lower())
    
    # 2. A gente filtra manualmente (Limpeza)
    lemmas_limpos = [
        token.lemma_ for token in doc 
        if token.text not in lista_stopwords  # Tira stopwords
        and token.is_alpha                    # Tira pontuação (exclamações)
    ]
    
    return " ".join(lemmas_limpos)

# Aplicando a Mágica
df['texto_final'] = df['texto'].apply(processamento_completo)

print("--- O TEXTO PERFEITO PARA IA ---")
display(df[['texto', 'texto_final']])

--- O TEXTO PERFEITO PARA IA ---


,texto,texto_final
0,O produto chegou quebrado e a caixa rasgada!!!,produto chegar quebrar caixa rasgar
1,"Amei muito, entrega super rápida e segura.",ameir entregar super rápido seguro
2,"Péssimo atendimento, nunca mais compro.",péssimo atendimento nunca compro
3,"O produto é bom, mas a entrega demorou.",produto bom entrega demorar


### NLTK vs Spacy no Fluxo

Embora ambas sejam bibliotecas de PLN, no nosso código elas atuaram de formas muito diferentes. É comum usar as duas juntas para aproveitar o melhor de cada mundo.

#### 1. NLTK: "O Bibliotecário (Consultor)" 
No nosso fluxo moderno, o NLTK atuou como um **fornecedor de referência**.
*   **Papel:** Passivo.
*   **Função:** Nós não usamos o NLTK para processar o texto pesado. Nós o usamos para **consultar a lista de Stopwords**.
*   **Analogia:** O Spacy pergunta: *"Ei NLTK, 'para' é uma palavra importante?"*. O NLTK consulta seu livro e diz: *"Não, pode jogar fora"*.

#### 2. Spacy: "O Chef (Processador)" 
Ele é quem coloca a mão na massa e transforma o ingrediente.
*   **Papel:** Ativo.
*   **Função:** Ele lê o texto, entende a gramática, identifica verbos e converte para o infinitivo (Lemma). É a biblioteca mais "pesada" e inteligente do fluxo.
*   **Analogia:** Ele pega a "batata suja" (texto bruto), descasca, corta e devolve a "batata pronta" (lemma).

---

#### Tabela Comparativa (Neste Código)

| Característica | NLTK | Spacy |
| :--- | :--- | :--- |
| **Atuação** | **Passiva** (Forneceu a lista de palavras proibidas) | **Ativa** (Leu, interpretou e transformou o texto) |
| **Velocidade** | **Rápido** (Apenas uma consulta de lista) | **Lento** (Carrega redes neurais para entender gramática) |
| **Resultado** | Deu insumo (`stopwords`) | Deu o produto final (`lemmas`) |

> **Resumo:** O **Pandas** segurou o texto para o **Spacy** processar. O Spacy, enquanto processava, pediu ajuda ao **NLTK** para saber o que jogar fora.